In [42]:
import os
import pickle

import numpy as np
from matplotlib import pyplot as plt

import pandas as pd
import seaborn as sns

In [43]:
stat_test_name = "test_cifar_pref"

stat_tests_path = "stat_tests" 
stat_test_path = os.path.join(stat_tests_path, stat_test_name)
if not os.path.exists(stat_test_path):
    raise FileNotFoundError(f"Stat test path {stat_test_path} does not exist.")

init_params_path = os.path.join(stat_test_path, "init_params.pkl")
test_params_path = os.path.join(stat_test_path, "test_params.pkl")

with open(init_params_path, "rb") as f:
    init_params = pickle.load(f)
with open(test_params_path, "rb") as f:
    test_params = pickle.load(f)


print("Init params:")
for k, v in init_params.items():
    print(f"{k}: {v}")

num_tests = init_params["num_tests"] 
num_tests = 10

tests_results = []  # 2-dim list [idx of repeated test][idx of test]
for i in range(num_tests):
    test_path = os.path.join(stat_test_path, f"test_{i}")
    test_result_path = os.path.join(test_path, "test_results.pkl")
    
    with open(test_result_path, "rb") as f:

        test_result = pickle.load(f)
    tests_results.append(test_result)

Init params:
test_name: test_cifar_pref
dataset_name: cifar10
num_clients: 5
num_classes: 10
distribution_type: preferential_class
model_name: resnet18
loss_name: cross_entropy
trainer_name: sgd
train_epochs: 40
target_client: 0
num_tests: 20


In [44]:
try:
    for i in range(num_tests):
        for j in range(len(tests_results[i])):
            tests_results[i][j]['trained_target_accuracy'] = tests_results[i][j]['trained_clients_accuracies'][init_params['target_client']]
            tests_results[i][j]['benchmark_target_accuracy'] = tests_results[i][j]['benchmark_clients_accuracies'][init_params['target_client']]
            tests_results[i][j]['reset_target_accuracy'] = tests_results[i][j]['reset_clients_accuracies'][init_params['target_client']]
            tests_results[i][j]['retrained_target_accuracy'] = tests_results[i][j]['retrained_clients_accuracies'][init_params['target_client']]
    
    print("Target from clients")
except:
    pass

try:
    for i in range(num_tests):
        for j in range(len(tests_results[i])):
            tests_results[i][j]['trained_target_accuracy'] = tests_results[i][j]['trained_class_accuracies'][init_params['target_client']+init_params['num_clients']]
            tests_results[i][j]['benchmark_target_accuracy'] = tests_results[i][j]['benchmark_class_accuracies'][init_params['target_client']+init_params['num_clients']]
            tests_results[i][j]['reset_target_accuracy'] = tests_results[i][j]['reset_class_accuracies'][init_params['target_client']+init_params['num_clients']]
            tests_results[i][j]['retrained_target_accuracy'] = tests_results[i][j]['retrained_class_accuracies'][init_params['target_client']+init_params['num_clients']]
    
    print("Target from classes")
except:
    pass


Target from classes


In [45]:
subtest = 1

subtest_params = []
subtest_results = []    # 2-dim list [idx of test][idx of repeated test]
for i, test_param in enumerate(test_params):
    if test_param["subtest"] == subtest:
        subtest_params.append(test_param)
        subtest_results.append([])
        for j in range(num_tests):
            subtest_results[-1].append(tests_results[j][i])

print("Subtest params:")
for key, value in subtest_params[0].items():
    if key == "unlearning_percentage":
        continue
    print(f"{key}: {value}")

print(f"\nAvailable results:")
for key in subtest_results[0][0].keys():
    print(key)

percentages = []
for test_param in subtest_params:
    percentages.append(int(test_param["unlearning_percentage"]))

print(f"\nUnlearning percentages: {percentages}")

Subtest params:
subtest: 1
unlearning_method: parameters
tests: ['test_accuracy', 'class_accuracies', 'mia']
mia_classifier_types: ['nn', 'logistic']
retrain_epochs: 1

Available results:
total_individual_reset_params
trained_test_accuracy
benchmark_test_accuracy
reset_test_accuracy
retrained_test_accuracy
trained_class_accuracies
benchmark_class_accuracies
reset_class_accuracies
retrained_class_accuracies
trained_mia_nn
benchmark_mia_nn
reset_mia_nn
retrained_mia_nn
trained_mia_logistic
benchmark_mia_logistic
reset_mia_logistic
retrained_mia_logistic
trained_target_accuracy
benchmark_target_accuracy
reset_target_accuracy
retrained_target_accuracy

Unlearning percentages: [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75]


In [46]:
percentage = 40

mia_method = "logistic"

selected_idx = percentages.index(percentage)
selected_results = subtest_results[selected_idx]

data= { 'Model': ['trained', 'benchmark', 'reset', 'retrained'],
        'Target accuracy': [],
        'MIA': [],
        'Test accuracy': []
        }

for model in data["Model"]:
    target_accuracies = [result[f'{model}_target_accuracy'] for result in selected_results]
    test_accuracies = [result[f'{model}_test_accuracy'] for result in selected_results]
    mia_aucs = [result[f'{model}_mia_{mia_method}']['roc_auc'] for result in selected_results]
    target_avg = np.mean(target_accuracies)*100
    target_std = np.std(target_accuracies)*100
    test_avg = np.mean(test_accuracies)*100
    test_std = np.std(test_accuracies)*100
    mia_avg = np.mean(mia_aucs)*100
    mia_std = np.std(mia_aucs)*100

    data["Target accuracy"].append(f"{target_avg:.2f} ± {target_std:.2f}")
    data["MIA"].append(f"{mia_avg:.2f} ± {mia_std:.2f}")
    data["Test accuracy"].append(f"{test_avg:.2f} ± {test_std:.2f}")

df = pd.DataFrame(data)
df = df.rename(columns={"MIA": f"MIA ({mia_method})"})
caption = f"Results for {stat_test_name} with unlearning percentage {percentage}%"
tex_escape = {
    '&':  '\&',
    '%':  '\%', 
    '$':  '\$', 
    '#':  '\#', 
    '_':  '\_', 
    '^':  '\^', 
}
for key, value in tex_escape.items():
    caption = caption.replace(key, value)
latex_table = df.to_latex(index=False, escape=False, caption=caption)
print(latex_table)

\begin{table}
\caption{Results for test\_cifar\_pref with unlearning percentage 40\%}
\begin{tabular}{llll}
\toprule
Model & Target accuracy & MIA (logistic) & Test accuracy \\
\midrule
trained & 68.28 ± 4.04 & 65.52 ± 1.89 & 79.75 ± 0.52 \\
benchmark & 0.00 ± 0.00 & 71.98 ± 1.25 & 73.41 ± 0.36 \\
reset & 0.00 ± 0.00 & 55.77 ± 7.02 & 10.91 ± 1.36 \\
retrained & 0.00 ± 0.00 & 70.94 ± 0.49 & 70.24 ± 0.38 \\
\bottomrule
\end{tabular}
\end{table}

